In [1]:
%load_ext autoreload
%autoreload 2

import logging
import json
import requests
import os
import pandas as pd

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

from datetime import datetime
from logging.handlers import TimedRotatingFileHandler


from krx.api import Krx_main_api
from krx.processing import upload_func
from krx.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect

now = datetime.now()

In [6]:
conf = get_jinja_yaml_conf('./conf/api.yml', './conf/logging.yml')
api_start_date = datetime.strptime(conf['api_start_date'], '%Y-%m-%d').date()
end_date = datetime.now().date() 


In [7]:
# Only for notebooks
import re


with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
krx_api = Krx_main_api(auth_key = os.environ['auth_key'])

In [11]:
conf['sto']['kospi']

{'kospi': {'detail_url': 'stk_bydd_trd',
  'rename': {'BAS_DD': '기준일자',
   'ISU_CD': '종목코드',
   'ISU_NM': '종목명',
   'MKT_NM': '시장구분',
   'SECT_TP_NM': '소속부',
   'TDD_CLSPRC': '종가',
   'CMPPREVDD_PRC': '대비',
   'FLUC_RT': '등락률',
   'TDD_OPNPRC': '시가',
   'TDD_HGPRC': '고가',
   'TDD_LWPRC': '저가',
   'ACC_TRDVOL': '거래량',
   'ACC_TRDVAL': '거래대금',
   'MKTCAP': '시가총액',
   'LIST_SHRS': '상장주식수'},
  'table_name': 'sto_stocks',
  'date_col': '기준일자',
  'class_col': '시장구분',
  'class_name': 'KOSPI',
  'skip_weekend': True},
 'kosdaq': {'detail_url': 'ksq_bydd_trd',
  'rename': {'BAS_DD': '기준일자',
   'ISU_CD': '종목코드',
   'ISU_NM': '종목명',
   'MKT_NM': '시장구분',
   'SECT_TP_NM': '소속부',
   'TDD_CLSPRC': '종가',
   'CMPPREVDD_PRC': '대비',
   'FLUC_RT': '등락률',
   'TDD_OPNPRC': '시가',
   'TDD_HGPRC': '고가',
   'TDD_LWPRC': '저가',
   'ACC_TRDVOL': '거래량',
   'ACC_TRDVAL': '거래대금',
   'MKTCAP': '시가총액',
   'LIST_SHRS': '상장주식수'},
  'table_name': 'sto_stocks',
  'date_col': '기준일자',
  'class_col': '시장구분',
  'class_name': '

In [4]:
conf['gen']

{'gold': {'detail_url': 'gold_bydd_trd',
  'rename': {'BAS_DD': '기준일자',
   'ISU_CD': '종목코드',
   'ISU_NM': '종목명',
   'TDD_CLSPRC': '종가',
   'CMPPREVDD_PRC': '대비',
   'FLUC_RT': '등락률',
   'TDD_OPNPRC': '시가',
   'TDD_HGPRC': '고가',
   'TDD_LWPRC': '저가',
   'ACC_TRDVOL': '거래량',
   'ACC_TRDVAL': '거래대금'},
  'table_name': 'gen_prod',
  'date_col': '기준일자',
  'class_col': '상장타입',
  'class_name': 'GOLD',
  'skip_weekend': True},
 'emit': {'detail_url': 'ets_bydd_trd',
  'rename': {'BAS_DD': '기준일자',
   'ISU_CD': '종목코드',
   'ISU_NM': '종목명',
   'TDD_CLSPRC': '종가',
   'CMPPREVDD_PRC': '대비',
   'FLUC_RT': '등락률',
   'TDD_OPNPRC': '시가',
   'TDD_HGPRC': '고가',
   'TDD_LWPRC': '저가',
   'ACC_TRDVOL': '거래량',
   'ACC_TRDVAL': '거래대금'},
  'table_name': 'gen_prod',
  'date_col': '기준일자',
  'class_col': '상장타입',
  'class_name': 'EMIT',
  'skip_weekend': True,
  'start_date': datetime.date(2015, 1, 13)},
 'oil': {'detail_url': 'oil_bydd_trd',
  'rename': {'BAS_DD': '기준일자',
   'OIL_NM': '유종구분',
   'WT_AVG_PRC': '가중평균

In [21]:
o = json.loads(requests.get(f"https://data-dbg.krx.co.kr/svc/apis/{'gen'}/{conf['gen']['emit']['detail_url']}?basDd=20240321", 
             headers ={
        'AUTH_KEY': os.environ['auth_key']
        }).text)['OutBlock_1']

o = pd.DataFrame(o)
o

,BAS_DD,ISU_CD,ISU_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL
0,20240321,05003230,KAU23,9020,-10,-0.11,9010,9020,8990,171801,1547239020
1,20240321,05003240,KAU24,8610,0,0.00,0,0,0,0,0
2,20240321,05003250,KAU25,8610,0,0.00,0,0,0,0,0
3,20240321,05103230,KCU23,7500,0,0.00,0,0,0,0,0
4,20240321,05103240,KCU24,7500,0,0.00,0,0,0,0,0
5,20240321,05203210,KOC21-26,11850,0,0.00,0,0,0,0,0
6,20240321,05203220,KOC22-27,11600,0,0.00,0,0,0,0,0
7,20240321,05203230,KOC23-28,14500,0,0.00,0,0,0,0,0
8,20240321,05203240,KOC24-29,11850,0,0.00,0,0,0,0,0
9,20240321,05303230,i-KCU23,15450,0,0.00,0,0,0,0,0


In [20]:
f"https://data-dbg.krx.co.kr/svc/apis/{'gen'}/{conf['gen']['emit']['detail_url']}?basDd=20240320"

'https://data-dbg.krx.co.kr/svc/apis/gen/ets_bydd_trd?basDd=20240320'